In [1]:
import pandas as pd
import re

In [2]:
requirements_df = pd.read_csv('key_words_requirements.csv', sep=';')
requirements_df.columns = ['ID_PYTANIA', 'PYTANIE', 'KLUCZ', 'ID_SEKCJA']

In [3]:
requirements_df

,ID_PYTANIA,PYTANIE,KLUCZ,ID_SEKCJA
0,1,Czy SFCR zawiera informacje dotyczące nazwy i ...,nazwa i forma prawna; Towarzystwo; Zakład; TU;...,3
1,2,Czy SFCR zawiera informacje dotyczące nazwy i ...,organ nadzoru; nadzór nad grupą,3
2,3,Czy SFCR zawiera informacje dotyczące nazwy or...,biegły rewident; audytor; audyt,3
3,4,Czy SFCR zawiera informacje dotyczące udziałow...,udziałowiec,3
4,5,Czy SFCR zawiera informacje dotyczące nazwy i ...,pośredni; bezpośredni; udziałowiec,3
...,...,...,...,...
162,163,"Czy SFCR, w odniesieniu do każdego przypadku n...",niezgodności z minimalnym wymogiem kapitałowym...,36
163,164,"Czy SFCR, w przypadku gdy niezgodność z minima...",niezgodność; kwota,36
164,165,Czy SFCR w odniesieniu do każdego istotnego pr...,niezgodności z kapitałowym wymogiem wypłacalno...,36
165,166,"Czy SFCR, w przypadku gdy niezgodność z kapita...",niezgodność; kwota,36


In [13]:
content_df = pd.read_csv('key_words_sample.csv',sep=';')
content_df.columns = ['ID_TAB', 'DATA SFCR', 'WERSJA SFCR', 'KOD ZAKŁADU', 'ID_SEKCJA NADRZĘDNA', 'ID_SEKCJA', 'NAZWA_SECKJI', 'TREŚĆ']

columns_to_fill = ['ID_TAB']
content_df[columns_to_fill] = content_df[columns_to_fill].ffill()

In [15]:
content_df

,ID_TAB,DATA SFCR,WERSJA SFCR,KOD ZAKŁADU,ID_SEKCJA NADRZĘDNA,ID_SEKCJA,NAZWA_SECKJI,TREŚĆ
0,1.0,31.12.2022,1.0,6.0,NaN,NaN,A. Działalność Spółki i wyniki,NaN
1,1.0,NaN,NaN,NaN,NaN,3.0,A.1. Działalność,"""Aegon Towarzystwo Ubezpieczeń na Życ"
2,1.0,NaN,NaN,NaN,NaN,3.0,A.1.1. Informacje ogólne,Aegon Towarzystwo Ubezpieczeń na Życie S.A. je...
3,1.0,NaN,NaN,NaN,NaN,3.0,A.1.2. Informacje ogólne,Aegon Towarzystwo Ubezpieczeń na Życie S.A. je...
4,1.0,NaN,NaN,NaN,NaN,4.0,A.2. Wyniki z działalności ubezpieczeniowej,Wyniki z działalności ubezpieczeniowej zostały...
5,1.0,NaN,NaN,NaN,NaN,9.0,B.1. Informacje ogólne,System Zarządzania w ocenie Towarzystwa jest a...


In [16]:
def has_more_than_two_uppercase_letters(word):
    return sum(1 for char in word if char.isupper()) > 2

def count_key_phrases(text, key_phrases):
    words = text.split()
    key_phrase_counts = {} 
    for phrase in key_phrases:
        # idea: if some word has more than 2 uppercase letters, search it case-sensitive, otherwise case-insensitive
        split_phrase = phrase.split()
        split_phrase_length = len(split_phrase)
        words_length = len(words)

        if any(has_more_than_two_uppercase_letters(word) for word in split_phrase):
            count = sum(phrase == ' '.join(words[i:i + split_phrase_length])
                        for i in range(words_length - split_phrase_length + 1))
        else:
            count = sum(phrase.lower() == ' '.join(words[i:i + split_phrase_length]).lower()
                        for i in range(words_length - split_phrase_length + 1))
        
        key_phrase_counts[phrase] = count

    return key_phrase_counts


In [19]:
def search_key_words(requirements_df, content_df, id_question, id_tab):
    key_phrases  = [phrase.strip() for  phrase in requirements_df[(requirements_df['ID_PYTANIA'] == id_question)]['KLUCZ'].values[0].split(';')]
    content_df = content_df[content_df['ID_TAB'] == id_tab] 
    sfcr_date = content_df['DATA SFCR'].values[0]
    sfcr_version = content_df['WERSJA SFCR'].values[0]
    company_code = content_df['KOD ZAKŁADU'].values[0]

    result_dict = {
        "ID_TAB" : id_tab,
        'DATA SFCR' : sfcr_date,
        'WERSJA SFCR' : sfcr_version,
        'KOD ZAKŁADU' : company_code,
        'ID_PYTANIA' : id_question
    }

    section_id = requirements_df[(requirements_df['ID_PYTANIA'] == id_question)]['ID_SEKCJA'].values[0]
    content = content_df[(content_df['ID_SEKCJA'] == section_id)]['TREŚĆ']
    # join sections content and clean it
    joined_content = ' '.join(list(content)).replace('\n', ' ')
    text = re.sub(r'[^a-zA-Z0-9ąćęłńóśźżĄĆĘŁŃÓŚŹŻ\s]', '', joined_content)
    key_phrase_counts = count_key_phrases(text, key_phrases)
    result = sum(key_phrase_counts.values())

    result_dict['CZY WYSTĄPIŁ KLUCZ'] = 1 if result > 0 else 0
    result_dict['ILOŚĆ WYSTĄPIEŃ KLUCZY'] = result

    return result_dict


In [20]:
id_tab = 1
id_question = 1

search_key_words(requirements_df, content_df, id_question, id_tab)

{'ID_TAB': 1,
 'DATA SFCR': '31.12.2022',
 'WERSJA SFCR': 1.0,
 'KOD ZAKŁADU': 6.0,
 'ID_PYTANIA': 1,
 'CZY WYSTĄPIŁ KLUCZ': 1,
 'ILOŚĆ WYSTĄPIEŃ KLUCZY': 7}

In [29]:
id_tab = 1
id_question = 2

search_key_words(requirements_df, content_df, id_question, id_tab)

{'ID_TAB': 1,
 'DATA SFCR': '31.12.2022',
 'WERSJA SFCR': 1.0,
 'KOD ZAKŁADU': 6.0,
 'ID_PYTANIA': 2,
 'CZY WYSTĄPIŁ KLUCZ': 0,
 'ILOŚĆ WYSTĄPIEŃ KLUCZY': 0}

In [28]:
id_tab = 1
id_question = 11

search_key_words(requirements_df, content_df, id_question, id_tab)

{'ID_TAB': 1,
 'DATA SFCR': '31.12.2022',
 'WERSJA SFCR': 1.0,
 'KOD ZAKŁADU': 6.0,
 'ID_PYTANIA': 11,
 'CZY WYSTĄPIŁ KLUCZ': 1,
 'ILOŚĆ WYSTĄPIEŃ KLUCZY': 10}

In [32]:
id_tab = 1
id_question = 21

search_key_words(requirements_df, content_df, id_question, id_tab)

id_tab = 1
id_question = 20

search_key_words(requirements_df, content_df, id_question, id_tab)

{'ID_TAB': 1,
 'DATA SFCR': '31.12.2022',
 'WERSJA SFCR': 1.0,
 'KOD ZAKŁADU': 6.0,
 'ID_PYTANIA': 20,
 'CZY WYSTĄPIŁ KLUCZ': 0,
 'ILOŚĆ WYSTĄPIEŃ KLUCZY': 0}